# vi älskar choklad!

- dont use REF (review index) and Review Date
- target: Rating
- labes: Company, Company Location, Bean Type, Broad Bean Origin, Specific Bean Origin, Cacao Percentage

In [1]:
import pandas as pd
from sklearn import preprocessing
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
from OSMPythonTools.nominatim import Nominatim
from tqdm import tqdm
tqdm.pandas()
import functools
import math
import numpy as np
import logging
logging.getLogger('OSMPythonTools').setLevel(logging.ERROR)

In [2]:
df = pd.read_csv('flavors_of_cacao.csv', converters={'Cocoa Percent': lambda x: float(x.strip('%'))/100})
df = df.drop(columns=['REF', 'Review Date'])

df

,Company,Specific Bean Origin,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,A. Morin,Agua Grande,0.63,France,3.75,,Sao Tome
1,A. Morin,Kpime,0.70,France,2.75,,Togo
2,A. Morin,Atsane,0.70,France,3.00,,Togo
3,A. Morin,Akata,0.70,France,3.50,,Togo
4,A. Morin,Quilla,0.70,France,3.50,,Peru
...,...,...,...,...,...,...,...
1790,Zotter,Peru,0.70,Austria,3.75,,Peru
1791,Zotter,Congo,0.65,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,0.65,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,0.62,Austria,3.25,,India


In [3]:
nominatim = Nominatim()
def beanlocation(row, key='display_namee'):
    loc = nominatim.query('{}, {}'.format(row['Specific Bean Origin'], row['Broad Bean Origin'])).toJSON()
    if len(loc) == 0:
        loc = nominatim.query(row['Broad Bean Origin']).toJSON()
    if len(loc) == 0:
        return None
    return loc[0][key]
        
    
df.loc[:,'bean_lat'] = df.progress_apply(beanlocation, key='lat', axis=1)
df.loc[:,'bean_lon'] = df.progress_apply(beanlocation, key='lon', axis=1)

100%|██████████| 1795/1795 [00:00<00:00, 1797.39it/s]


In [4]:
def complocation(row, key='display_namee'):
    loc = nominatim.query(row['Company Location']).toJSON()
    if len(loc) == 0:
        return None
    return loc[0][key]
        
    
df.loc[:,'comp_lat'] = df.progress_apply(complocation, key='lat', axis=1)
df.loc[:,'comp_lon'] = df.progress_apply(complocation, key='lon', axis=1)

100%|██████████| 1795/1795 [00:00<00:00, 2609.79it/s]


In [5]:
print('deleting {} rows (no coordinates found)'.format(((df['bean_lat'].to_numpy() == None) | (df['comp_lat'].to_numpy() == None)).sum()))
df = df[(df['bean_lat'].to_numpy() != None) & (df['comp_lat'].to_numpy() != None)]

deleting 86 rows (no coordinates found)


In [6]:
df

,Company,Specific Bean Origin,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin,bean_lat,bean_lon,comp_lat,comp_lon
0,A. Morin,Agua Grande,0.63,France,3.75,,Sao Tome,0.3389242,6.7313031,46.603354,1.8883335
1,A. Morin,Kpime,0.70,France,2.75,,Togo,6.9734449,0.649697,46.603354,1.8883335
2,A. Morin,Atsane,0.70,France,3.00,,Togo,8.7800265,1.0199765,46.603354,1.8883335
3,A. Morin,Akata,0.70,France,3.50,,Togo,7.0501355,0.7099619,46.603354,1.8883335
4,A. Morin,Quilla,0.70,France,3.50,,Peru,-13.6951583,-74.0586314,46.603354,1.8883335
...,...,...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,0.70,Austria,3.75,,Peru,-6.8699697,-75.0458515,47.69641095,14.754807384371395
1791,Zotter,Congo,0.65,Austria,3.00,Forastero,Congo,-0.7179165000000001,16.018048574081842,47.69641095,14.754807384371395
1792,Zotter,Kerala State,0.65,Austria,3.50,Forastero,India,9.5629176,76.7622342,47.69641095,14.754807384371395
1793,Zotter,Kerala State,0.62,Austria,3.25,,India,9.5629176,76.7622342,47.69641095,14.754807384371395


In [7]:
companyle = preprocessing.LabelEncoder()
df.loc[:,'company_id'] = companyle.fit_transform(df.loc[:,'Company'])
beantypele = preprocessing.LabelEncoder()
df.loc[:,'beantype_id'] = beantypele.fit_transform(df.loc[:,'Bean Type'])

/home/marcel/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/marcel/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
df

,Company,Specific Bean Origin,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin,bean_lat,bean_lon,comp_lat,comp_lon,company_id,beantype_id
0,A. Morin,Agua Grande,0.63,France,3.75,,Sao Tome,0.3389242,6.7313031,46.603354,1.8883335,0,39
1,A. Morin,Kpime,0.70,France,2.75,,Togo,6.9734449,0.649697,46.603354,1.8883335,0,39
2,A. Morin,Atsane,0.70,France,3.00,,Togo,8.7800265,1.0199765,46.603354,1.8883335,0,39
3,A. Morin,Akata,0.70,France,3.50,,Togo,7.0501355,0.7099619,46.603354,1.8883335,0,39
4,A. Morin,Quilla,0.70,France,3.50,,Peru,-13.6951583,-74.0586314,46.603354,1.8883335,0,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,0.70,Austria,3.75,,Peru,-6.8699697,-75.0458515,47.69641095,14.754807384371395,402,39
1791,Zotter,Congo,0.65,Austria,3.00,Forastero,Congo,-0.7179165000000001,16.018048574081842,47.69641095,14.754807384371395,402,19
1792,Zotter,Kerala State,0.65,Austria,3.50,Forastero,India,9.5629176,76.7622342,47.69641095,14.754807384371395,402,19
1793,Zotter,Kerala State,0.62,Austria,3.25,,India,9.5629176,76.7622342,47.69641095,14.754807384371395,402,39


In [9]:
xlabels = ['Cocoa Percent', 'beantype_id', 'company_id', 'comp_lat', 'comp_lon', 'bean_lat', 'bean_lon']
df[xlabels] # X

,Cocoa Percent,beantype_id,company_id,comp_lat,comp_lon,bean_lat,bean_lon
0,0.63,39,0,46.603354,1.8883335,0.3389242,6.7313031
1,0.70,39,0,46.603354,1.8883335,6.9734449,0.649697
2,0.70,39,0,46.603354,1.8883335,8.7800265,1.0199765
3,0.70,39,0,46.603354,1.8883335,7.0501355,0.7099619
4,0.70,39,0,46.603354,1.8883335,-13.6951583,-74.0586314
...,...,...,...,...,...,...,...
1790,0.70,39,402,47.69641095,14.754807384371395,-6.8699697,-75.0458515
1791,0.65,19,402,47.69641095,14.754807384371395,-0.7179165000000001,16.018048574081842
1792,0.65,19,402,47.69641095,14.754807384371395,9.5629176,76.7622342
1793,0.62,39,402,47.69641095,14.754807384371395,9.5629176,76.7622342


In [10]:
ylabel = 'Rating'
df[ylabel]

0       3.75
1       2.75
2       3.00
3       3.50
4       3.50
        ... 
1790    3.75
1791    3.00
1792    3.50
1793    3.25
1794    3.00
Name: Rating, Length: 1709, dtype: float64

In [11]:
#shuffel
df = df.sample(frac=1, random_state=1337).reset_index(drop=True)

In [12]:
#split
i = math.floor(df.shape[0] * .8)
train,test = df.iloc[:i], df.iloc[i:]

In [13]:
train_x = np.ascontiguousarray(train[xlabels].to_numpy())
train_y = np.ascontiguousarray(train[ylabel].to_numpy())
test_x = np.ascontiguousarray(test[xlabels].to_numpy())
test_y = np.ascontiguousarray(test[ylabel].to_numpy())

In [14]:
train_x = preprocessing.maxabs_scale(train_x)
test_x = preprocessing.maxabs_scale(test_x)